## Getting Data from Firebase

In [90]:
from flask import Flask, render_template, request
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import pandas as pd
import numpy as np
from datetime import datetime

In [91]:
cred = credentials.Certificate("finalkey.json")
firebase_admin.initialize_app(cred, {'databaseURL': 'https://final-project-f23d0-default-rtdb.firebaseio.com/'})

In [109]:
def get_values(name):
    ref = db.reference('/')
    temp_sensor = ref.child(f'climate/{name}').get()

    timestamp = []
    value = []
    
    for key, val in temp_sensor.items():
        value.append(val[name])
        timestamp.append(val['timestamp'])

    return timestamp, value

def viz(timestamp, value, name):
    data = go.Scatter(x=timestamp, y=value)
    fig = go.Figure(data=data)

    fig.layout.title = f'{name} Over Day'
    fig.layout.xaxis.title = 'Timestamp'
    fig.layout.yaxis.title = name

    return fig

def all_viz(timestamp, temp, humid, photo):
    trace1 = go.Scatter(x=timestamp, y=temp, mode='lines', name='Temperature')
    trace2 = go.Scatter(x=timestamp, y=humid, mode='lines', name='Humidity')
    trace3 = go.Scatter(x=timestamp, y=photo, mode='lines', name='Photoresistor')

    fig = go.Figure()
    fig.add_trace(trace1)
    fig.add_trace(trace2)
    fig.add_trace(trace3)

    fig.update_layout(
        title='Climate Over Time',
        xaxis_title='Timestamp',
        yaxis_title='Value'
    )

    return fig

def Predict_viz():
    # Create a scatter plot
    trace = go.Scatter(x=y_test, y=y_pred,
        mode='markers',
        marker=dict(color='blue'),
        name='Actual vs Predicted'
    )

    # Add the ideal line
    ideal_line = go.Scatter(
        x=[min(y_test), max(y_test)],
        y=[min(y_test), max(y_test)],
        mode='lines',
        line=dict(color='red', dash='dash'),
        name='Ideal Line'
    )

    # Layout
    layout = go.Layout(
        title='Actual vs Predicted Temperature',
        xaxis=dict(title='Actual Temperature'),
        yaxis=dict(title='Predicted Temperature'),
        showlegend=True
    )

    # Combine trace and layout
    fig = go.Figure(data=[trace, ideal_line], layout=layout)

    return fig

In [110]:
ref = db.reference('/')

timestamp, temp = get_values('temp')
tempFig = viz(timestamp, temp, 'Temperature').to_json()

_, humid = get_values('humid')
HumidFig = viz(timestamp, humid, 'Humidity').to_json()

_, photo = get_values('photo')
PhotoFig = viz(timestamp, photo, 'Photoresistor').to_json()

allFig = all_viz(timestamp, temp, humid, photo).to_json()
PredictFig = Predict_viz().to_json()

In [94]:
# Convert timestamps to numerical representation
start_time = datetime.strptime(timestamp[0], "%Y-%m-%dT%H:%M:%S.%f")
time_numeric = [(datetime.strptime(ts, "%Y-%m-%dT%H:%M:%S.%f") - start_time).total_seconds() if ts is not None else None for ts in timestamp]

humid = np.array(humid, dtype=float)
temp = np.array(temp, dtype=float)
photo = np.array(photo, dtype=float)
time_numeric = np.array(time_numeric, dtype=float)

# Handle None values in humidity by replacing them with the mean
humid_mean = np.nanmean(humid)
humid = np.where(np.isnan(humid), humid_mean, humid)

# Handle None values in photo by replacing them with the mean
photo_mean = np.nanmean(photo)
photo = np.where(np.isnan(photo), photo_mean, photo)

# Handle None values in Temperature by replacing them with the mean
temp_mean = np.nanmean(temp)
temp = np.where(np.isnan(temp), temp_mean, temp)

In [95]:
X = np.column_stack((time_numeric, humid, temp, photo))

In [96]:
X_train, X_test, y_train, y_test = train_test_split(X, temp, test_size=0.2, random_state=42)

In [97]:
# Create and train the linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [ ]:
app = Flask(__name__)

@app.route("/")
def home():
  return render_template("home.html", allFig=allFig)

@app.route("/humid")
def humid():
  return render_template("humid.html", HumidFig=HumidFig)

@app.route("/temp")
def temp():
  return render_template("temp.html", tempFig=tempFig)

@app.route("/photo")
def photo():
  return render_template("photo.html", PhotoFig=PhotoFig)

@app.route('/predict', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        # Get user input for time
        input_time = request.form['input_time']

        # Convert input time to datetime object
        input_datetime = datetime.strptime(input_time, '%Y-%m-%dT%H:%M')

        # Calculate elapsed seconds since the first timestamp
        input_time_numeric = (input_datetime - start_time).total_seconds()

        # Make prediction for temperature
        # input_data = np.array([[input_time_numeric, humidity_mean, temp[0], photo_mean]])
        # predicted_temperature = model.predict(input_data)[0]
        
        predicted_temperature = 5
        
        # Render template with prediction result
        return render_template('predict.html', PredictFig=PredictFig, input_time=input_time, predicted_temperature=predicted_temperature)
    
    # Render default template for GET requests
    return render_template('predict.html', PredictFig=PredictFig)


if __name__ == "__main__":
  app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [06/May/2024 17:04:46] "POST /predict HTTP/1.1" 200 -
